# Primary Gas Fluid Properties Fits `post_process.ipynb`

Suppose we have measurements for temperature pressure, and want to determine an unknown fluid property $Y$. We will measure these values within a specific range:

Fluid temperature, $T\in[10, 30]$  °C 

Fluid pressure, $P\in[0.95, 120]$ bar(a)

## 1 Transform Features

To enhance the model's ability to represent an unknown fluid property effectively, we map the measurands into a new vector space. This transformation involves converting the input vector $X$ to a polynomial $n^{th}$ degree surface form, denoted as $\Phi$.

Given the vector $X = [T, P]$, where $T$ and $P$ represent temperature and pressure respectively, the transformation can be described by:

$$
X \mapsto \Phi(T, P)
$$

$$
\Phi(T, P) = \{ T^i P^j \mid i \geq 0, j \geq 0, i + j \leq n \}
$$

Here, $X$ belongs to $\mathbb{R}^2$ and $\Phi$ to $\mathbb{R}^{\binom{2+n}{n}}$, reflecting the increased dimensionality of the feature space post-transformation.

The optimal selection of the polynomial degree $n$ for the model is determined using a hyperparameter grid search method or a similar approach.

## 2 Regression Model

Once we have transformed our measurands into the new feature space using $\Phi(T, P)$, the next step is to establish a relationship between these features and the unknown fluid property, $Y$. This relationship is modeled through a linear regression approach.

We seek to find a vector of coefficients $\vec{\beta} \in \mathbb{R}^{\binom{2+n}{n}}$, which best maps the functional form $\Phi(T, P)$ to $Y$. The model can be expressed as:

$$
Y = \Phi(T,P) \vec{\beta} + \epsilon
$$

Here, $\epsilon$ represents some unknown error, arising from the assumptions inherent in the model selection and the limitations of the linear approach. This error term captures the discrepancy between the predicted and actual values of $Y$.

## 3 Fluid Density

To enhance the accuracy of density calculations beyond what is achievable with a naive linear regression model, we consider splitting the fluid density into two components. The first component is the ideal gas density, $\rho_{ideal}$, and the second accounts for deviations from ideality using the compressibility factor, $Z$.

The ideal gas density is defined by the equation:
$$
\rho_{ideal} = \frac{P}{RT}
$$
where $P$ is the pressure, $T$ is the temperature, and $R$ is the specific gas constant.

However, since we do not have direct measurements of $Z$, we use the polynomial model $\Phi(T,P)\vec{\beta}$ to approximate $\hat{Z}\approx Z$.

With $\hat{Z}$ estimated, we can then approximate the fluid density, accounting for non-ideal behavior, as follows:
$$
\hat{\rho} = \frac{P}{RT} \cdot \left(\Phi(T,P) \vec{\beta}\right)^{-1}
$$

This formulation allows us to estimate the density as a function of pressure, temperature, and the specific gas constant, while incorporating adjustments based on the calculated compressibility factor from our model.

In [1]:
"""Density Fit Example Code with inputs for temperature and pressure in degrees Celsius and bar(a), respectively."""
from fit_fluid_properties import fit_gas_density

params = {
    "reference_data_directory": "data",
    "reference_data_filename": "H2 Nist Properties 0.95-120 bar.xlsx",
    "reference_fluid": "H2",
    "model_poly_degree": 4,
}

df_density, functional_form_text_density = fit_gas_density(params)

print(functional_form_text_density)
df_density.describe()

P * 10**5 / 4124.478823 / (T + 273.15) / (1.000026309822635 + 0.0*T**0*P**0 + -1.617275926517221e-06*T**1*P**0 + 0.0006140911119221944*T**0*P**1 + 6.521158300476665e-08*T**2*P**0 + -1.1293193508753775e-06*T**1*P**1 + 2.275536938872253e-07*T**0*P**2 + -5.125297349687858e-09*T**3*P**0 + 2.112098293632951e-09*T**2*P**1 + -4.2141128622948314e-09*T**1*P**2 + 5.5108648885280046e-11*T**0*P**3 + 1.4246155890301237e-10*T**4*P**0 + -8.029216697028959e-11*T**3*P**1 + 3.485366465820011e-11*T**2*P**2 + 9.173104337018004e-13*T**1*P**3 + -1.4068705963631142e-13*T**0*P**4)


,Temperature (C),Pressure (bar),Compressibility,Compressibility_hat,Density (kg/m3),Density_hat (kg/m3),Density Error (kg/m3),Density Relative Error (%)
count,2600.000000,2600.000000,2600.000000,2600.000000,2600.000000,2600.000000,2.600000e+03,2.600000e+03
mean,20.000000,58.652531,1.035476,1.035476,4.629043,4.629043,-5.584068e-10,-3.318263e-13
std,5.891345,35.573466,0.021754,0.021754,2.744573,2.744573,5.246122e-05,1.430097e-03
min,10.000000,0.950000,1.000546,1.000559,0.075938,0.075937,-2.199741e-04,-4.665867e-03
25%,14.898000,27.068500,1.015981,1.015964,2.165925,2.165968,-2.643458e-05,-7.641039e-04
50%,20.000000,58.045500,1.034878,1.034880,4.638850,4.638869,1.560638e-06,6.798746e-05
75%,25.102000,89.022500,1.054338,1.054329,7.030650,7.030680,3.265218e-05,7.329975e-04
max,30.000000,120.000000,1.075174,1.075166,9.556900,9.556963,1.595299e-04,7.181375e-03


## 4 Fluid Dynamic Viscosity

Similarly to the density calculation, to enhance the precision of dynamic viscosity calculations beyond what a naive linear regression model could offer, we separate the viscosity equation into two parts. The first is an idealized part calculated using Sutherland's Law, $\mu_{ideal}$, and the second is a compressible component calculated using a nonlinear function of temperature and pressure, $f(T, P)$.

The ideal part of the viscosity, $\mu_{ideal}$, is given by:
$$
\mu_{ideal} = \mu_0 \left(\frac{T}{T_0}\right)^{3/2} \frac{T_0 + S}{T + S}
$$
where $T_0$ is a reference temperature, $\mu_0$ is the reference viscosity at $T_0$, and $S$ is the Sutherland temperature for the fluid.

Given that we have measurements of $T$ and $P$, we can employ the polynomial model $\Phi(T, P)\vec{\beta}$ to approximate the compressible part of the equation:
$$
f(T, P) \approx \Phi(T, P)\vec{\beta}
$$

Consequently, the final approximation of the viscosity can be expressed as:
$$
\hat{\mu} = \mu_{ideal} \cdot f(T, P) = \mu_0 \left(\frac{T}{T_0}\right)^{3/2} \frac{T_0 + S}{T + S} \cdot \Phi(T, P)\vec{\beta}
$$

This formulation then allows us to estimate the dyanmic viscosity as a function of pressure, temperature, and known constants, while incorporating adjustments based on the real gas behaviours from our polynomial model.

In [2]:
"""Viscosity Fit Example Code with inputs for temperature and pressure in degrees Celsius and bar(a), respectively."""
from fit_fluid_properties import fit_gas_viscosity

params = {
    "reference_data_directory": "data",
    "reference_data_filename": "H2 Nist Properties 0.95-120 bar.xlsx",
    "reference_fluid": "H2",
    "model_poly_degree": 4,
}

df_viscosity, functional_form_text_viscosity = fit_gas_viscosity(params)

print(functional_form_text_viscosity)
df_viscosity.describe()

8.411e-06 * ((T + 273.15) / 273)**(3/2) * (273 + 97) / (T + 273.15 + 97) * (0.9954355357512115 + 0.0*T**0*P**0 + -0.00025019977047992584*T**1*P**0 + 8.480595001199402e-05*T**0*P**1 + 1.5030720160159363e-06*T**2*P**0 + -1.2255950736250391e-06*T**1*P**1 + 1.0328650453208853e-06*T**0*P**2 + 3.6469995559449026e-10*T**3*P**0 + 1.032784900143852e-08*T**2*P**1 + -8.00779577974134e-09*T**1*P**2 + -1.2677165928956245e-09*T**0*P**3 + -4.8249382105461114e-11*T**4*P**0 + -6.92711546146703e-11*T**3*P**1 + 2.9851814977802375e-11*T**2*P**2 + 9.813255134709375e-12*T**1*P**3 + 6.288808991790351e-13*T**0*P**4)


,Temperature (C),Pressure (bar),Viscosity (Pa*s),Viscosity_hat (Pa*s),Viscosity Error (Pa*s),Viscosity Relative Error (%)
count,2600.000000,2600.000000,2.600000e+03,2.600000e+03,2.600000e+03,2.600000e+03
mean,20.000000,58.652531,8.862255e-06,8.862255e-06,-7.022555e-21,1.074617e-09
std,5.891345,35.573466,1.292204e-07,1.292204e-07,2.905117e-11,3.278524e-04
min,10.000000,0.950000,8.588000e-06,8.588000e-06,-6.404334e-11,-7.199683e-04
25%,14.898000,27.068500,8.760000e-06,8.759972e-06,-2.376662e-11,-2.694835e-04
50%,20.000000,58.045500,8.863150e-06,8.863132e-06,3.441867e-13,3.893427e-06
75%,25.102000,89.022500,8.966425e-06,8.966396e-06,2.419613e-11,2.728499e-04
max,30.000000,120.000000,9.156600e-06,9.156617e-06,5.909278e-11,6.754464e-04


## 3 Fluid Isentropic Exponent

The fluid isentopic exponent, $\kappa$ is defined as the ratio of the specific heats at a constant pressure and constant volume, $C_p/C_v$.

For this model, we simply assume that this ratio can be expressed directly as a linear model of the temperature and pressures of the fluid expressed as a polynomial:

$$\hat\kappa=\Phi(T,P)\vec{\beta}$$

In [4]:
"""Isentropic Exponent Fit Example Code with inputs for temperature and pressure in degrees Celsius and bar(a), respectively."""

from fit_fluid_properties import fit_isentropic_exponent

params = {
    "reference_data_directory": "data",
    "reference_data_filename": "H2 Nist Properties 0.95-120 bar.xlsx",
    "reference_fluid": "H2",
    "model_poly_degree": 4,
}

df_isentropic_exponent, functional_form_text_isentropic_exponent = (
    fit_isentropic_exponent(params)
)

print(functional_form_text_isentropic_exponent)
df_isentropic_exponent.describe()

1.409418655355469 + 0.0*T**0*P**0 + -0.00016896096200970438*T**1*P**0 + 0.00021225955424609072*T**0*P**1 + -2.402870410273088e-06*T**2*P**0 + -2.173509972849281e-06*T**1*P**1 + -4.856309960985996e-07*T**0*P**2 + 1.4899052027301393e-07*T**3*P**0 + 2.783784176276097e-09*T**2*P**1 + 5.170269315935728e-09*T**1*P**2 + -4.6039706128373666e-10*T**0*P**3 + -2.116637348074477e-09*T**4*P**0 + 1.3163859447313457e-10*T**3*P**1 + -2.06278672494739e-11*T**2*P**2 + 1.9725523636910693e-12*T**1*P**3 + 1.9852576071781314e-12*T**0*P**4


,Temperature (C),Pressure (bar),Isentropic Exponent,Isentropic Exponent_hat,Isentropic Exponent Error,Isentropic Exponent Relative Error (%)
count,2600.000000,2600.000000,2600.000000,2600.000000,2.600000e+03,2.600000e+03
mean,20.000000,58.652531,1.414088,1.414088,-1.393757e-16,1.220608e-07
std,5.891345,35.573466,0.004602,0.004602,4.941426e-05,3.494400e-03
min,10.000000,0.950000,1.404629,1.404641,-1.152819e-04,-8.106392e-03
25%,14.898000,27.068500,1.410120,1.410119,-3.670700e-05,-2.598472e-03
50%,20.000000,58.045500,1.414399,1.414380,7.642404e-07,5.429765e-05
75%,25.102000,89.022500,1.417793,1.417772,3.547655e-05,2.508546e-03
max,30.000000,120.000000,1.423890,1.423901,1.175940e-04,8.280512e-03
